# Architecture conformance checking from logs
This notebook is what I used for Section 8.6 of my Master's thesis. It uses CSV files generated and converted by AJPOLog to create a dot file that represents the architecture. It uses an arbitrary mapping encoded in Python dicts to convert log entries into edges in a directed graph. The cells necessary for using this notebook are documented in the Markdown cells above them.

Requirements: Python 3.5+, pydot and networkx

The cell below is used for initalization and should always be ran.

In [10]:
import csv

unmapped = {} # make the scope of unmapped global

def architecture_map(structure):
    for m in mapping.keys():
        if structure.startswith(m):
            return mapping.get(m)
    # If it matches none of the classes in the mapping:
    if structure.startswith('net.sf.jabref'):
        return 'Global'
    unmapped.add(structure)
    return False

## Mapping cells
The two cells below each represent a mapping. They are dict representations of the mappings used in the thesis. The 3.7 mapping was created by us, and the SAEroCON mapping was converted from Tobias Dietz and Leo Pruijt.

In [6]:
# 3.7 mapping
mapping = {
    'net.sf.jabref.model': 'Data layer.model',
    'net.sf.jabref.preferences': 'Service layer.preferences',
    'net.sf.jabref.logic': 'Service layer.logic',
    'net.sf.jabref.cli': 'Interaction layer.cli',
    'net.sf.jabref.gui': 'Interaction layer.gui',
    'osx': 'osx',
    'oracle': 'oracle',
}

In [13]:
# SAEroCON mapping
mapping = {
    'net.sf.jabref.JabRefGUI': 'GUI',
    'net.sf.jabref.JabRefMain': 'GUI',
    'net.sf.jabref.collab.Change': 'GUI',
    'net.sf.jabref.collab.ChangeDisplayDialog': 'GUI',
    'net.sf.jabref.collab.ChangeScanner': 'GUI',
    'net.sf.jabref.collab.EntryAddChange': 'GUI',
    'net.sf.jabref.collab.EntryChange': 'GUI',
    'net.sf.jabref.collab.EntryDeleteChange': 'GUI',
    'net.sf.jabref.collab.FileUpdatePanel': 'GUI',
    'net.sf.jabref.collab.GroupChange': 'GUI',
    'net.sf.jabref.collab.InfoPane': 'GUI',
    'net.sf.jabref.collab.MetaDataChange': 'GUI',
    'net.sf.jabref.collab.PreambleChange': 'GUI',
    'net.sf.jabref.collab.StringAddChange': 'GUI',
    'net.sf.jabref.collab.StringChange': 'GUI',
    'net.sf.jabref.collab.StringNameChange': 'GUI',
    'net.sf.jabref.collab.StringRemoveChange': 'GUI',
    'net.sf.jabref.gui': 'GUI',
    'net.sf.jabref.migrations.FileLinksUpgradeWarning': 'GUI',
    'net.sf.jabref.pdfimport.ImportDialog': 'GUI',
    'net.sf.jabref.pdfimport.PdfFileFilter': 'GUI',
    'net.sf.jabref.pdfimport.PdfImporter': 'GUI',
    'net.sf.jabref.JabRefExecutorService': 'Logic',
    'net.sf.jabref.collab.FileUpdateListener': 'Logic',
    'net.sf.jabref.collab.FileUpdateMonitor': 'Logic',
    'net.sf.jabref.logic': 'Logic',
    'net.sf.jabref.shared.DBMSProcessor': 'Logic',
    'net.sf.jabref.shared.DBMSSynchronizer': 'Logic',
    'net.sf.jabref.shared.MySQLProcessor': 'Logic',
    'net.sf.jabref.shared.OracleProcessor': 'Logic',
    'net.sf.jabref.shared.PostgreSQLProcessor': 'Logic',
    'net.sf.jabref.shared.event': 'Logic',
    'net.sf.jabref.shared.exception': 'Logic',
    'net.sf.jabref.shared.listener': 'Logic',
    'net.sf.jabref.JabRefException': 'Model',
    'net.sf.jabref.model': 'Model',
    'net.sf.jabref.shared.DBMSConnection': 'Model',
    'net.sf.jabref.shared.DBMSConnectionProperties': 'Model',
    'net.sf.jabref.shared.DBMSType': 'Model',
    'net.sf.jabref.shared.security.Password': 'Model',
    'net.sf.jabref.Globals': 'Globals',
    'net.sf.jabref.cli': 'CommandLineInterface',
    'net.sf.jabref.migrations.PreferencesMigrations': 'Pref',
    'net.sf.jabref.preferences': 'Pref',
    'net.sf.jabref.shared.prefs': 'Pref',
    'java.awt': 'Swing/AWT',
    'javax.swing': 'Swing/AWT',
    'java.sql': 'SQL',
    'oracle': 'SQL'
}

## Graph conversion
The cell below converts a hardcoded file into a set of tuples that can be used for a graph. This is how the architecture diagrams were created. Change the filename to use your own file. The file must be a CSV file created by AJPOLog and converted using the conversion script in AJPOLog.

In [14]:
with open('D:/AJPOLog/logs/jabref-scenario-1-converted.csv') as f:
    reader = csv.DictReader(f, delimiter=";")
    unmapped = set()
    edges = set()
    for row in reader:
        caller = architecture_map(row['Caller'])
        callee = architecture_map(row['Callee'])
        if caller and callee:
            edges.add((caller, callee))

## Architecture violation detection
Creates a file with all log entries that violate the intended architecture but were not picked up by Pruijt and Dietz. It has limited uses, only detecting a specific type of architecture violation we found. This cell only makes sense to use together with the SAEroCON mapping. Again, filenames are hardcoded and need to be adapted before use.

In [15]:
with open('D:/AJPOLog/logs/jabref-scenario-1-converted.csv') as f:
    with open('D:/AJPOLog/logs/unexplained-violations.csv', 'w') as o:
        reader = csv.DictReader(f, delimiter=";")
        unmapped = set()
        edges = set()
        for row in reader:
            caller = architecture_map(row['Caller'])
            callee = architecture_map(row['Callee'])
            if caller == 'Pref' and callee == 'Logic':
                o.write(str(row))

## Test
This cell is used to test the output of the script. It should _not_ be empty when used after running the regular graph creation cell, but should be empty after using the cell above.

In [12]:
edges

{('CommandLineInterface', 'CommandLineInterface'),
 ('GUI', 'CommandLineInterface'),
 ('GUI', 'GUI'),
 ('GUI', 'Logic'),
 ('GUI', 'Model'),
 ('GUI', 'Pref'),
 ('GUI', 'Swing/AWT'),
 ('Globals', 'Logic'),
 ('Logic', 'GUI'),
 ('Logic', 'Logic'),
 ('Logic', 'Model'),
 ('Logic', 'Pref'),
 ('Model', 'Logic'),
 ('Model', 'Model'),
 ('Pref', 'GUI'),
 ('Pref', 'Logic'),
 ('Pref', 'Model'),
 ('Pref', 'Pref'),
 ('Swing/AWT', 'GUI')}

## Export to dotfile
Initially, we used a graphviz within Python, but this was rather cumbersome compared to copying the file into an online graphviz implementation. Editing the dotfile manually gave us greater freedom in how it should be displayed. We converted the output of graphviz to SVG which was then used within the thesis.

In [ ]:
import networkx as nx
import pydot
from networkx.drawing.nx_pydot import write_dot

g = nx.DiGraph()
g.add_edges_from(edges)

write_dot(g, "grid.dot")